# Models

> Networks

In [ ]:
# | default_exp models

In [ ]:
# | export

import torch.nn as nn
import torch.nn.functional as F
import torch

import numpy as np
import torch
import torch.nn as nn
from typing import List
import math
from neuralresonator.utilities import to_zpk
from neuralresonator.dsp import (constrain_complex_pole_or_zero,
                                 pole_or_zero_to_iir_coeff)
from torchinfo import summary


In [ ]:
# | export


class FCBlock(nn.Module):
    """
    A fully connected block with an activation function and optional layer normalization
    """

    def __init__(
        self,
        input_size: int,  # The size of the input
        output_size: int,  # The size of the output
        activation: nn.Module = nn.LeakyReLU(
            0.2, inplace=True
        ),  # The activation function
        layer_norm: bool = False,  # Whether to use layer normalization
    ):
        super().__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.activation = activation

        self.linear = nn.Linear(input_size, output_size)
        self.ln = nn.LayerNorm(output_size) if layer_norm else nn.Identity()

    def forward(
        self,
        x: torch.Tensor,
    ) -> torch.Tensor:
        return self.activation(self.ln(self.linear(x)))


class FC(nn.Module):
    """
    A fully connected network with an activation function and optional layer normalization
    """

    def __init__(
        self,
        input_size: int,  # The size of the input
        output_size: int,  # The size of the output
        hidden_sizes: List[int],  # The sizes of the hidden layers
        activation: nn.Module = nn.LeakyReLU(
            0.2, inplace=True
        ),  # The activation function
        layer_norm: bool = False,  # Whether to use layer normalization
    ):
        super().__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_sizes = hidden_sizes
        self.activation = activation

        layers = []

        # First layer
        layers.append(
            FCBlock(
                input_size,
                hidden_sizes[0],
                activation=activation,
                layer_norm=layer_norm,
            )
        )

        # Hidden layers
        for i in range(len(hidden_sizes)):
            # Set the input size equal to the output size
            n = hidden_sizes[i]

            layers.append(
                FCBlock(
                    hidden_sizes[i],
                    hidden_sizes[i],
                    activation=activation,
                    layer_norm=layer_norm,
                )
            )

        # Output layer
        layers.append(nn.Linear(hidden_sizes[-1], output_size))

        self.network = nn.Sequential(*layers)

    def forward(
        self,
        x: torch.Tensor,
    ) -> torch.Tensor:
        return self.network(x)


Test

In [ ]:

model = FC(
    input_size=7,
    output_size=100,
    hidden_sizes=[100, 100],
)

x = torch.randn(1, 7)
y = model(x)

assert y.shape == torch.Size([1, 100])
summary(model, input_size=(1, 7))


/home/diaz/anaconda3/envs/modal/lib/python3.10/site-packages/torchinfo/torchinfo.py:477: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  action_fn=lambda data: sys.getsizeof(data.storage()),
/home/diaz/anaconda3/envs/modal/lib/python3.10/site-packages/torch/storage.py:665: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return super().__sizeof__() + self.nbytes()


Layer (type:depth-idx)                   Output Shape              Param #
FC                                       [1, 100]                  --
├─Sequential: 1-1                        [1, 100]                  --
│    └─FCBlock: 2-1                      [1, 100]                  --
│    │    └─Linear: 3-1                  [1, 100]                  800
│    │    └─Identity: 3-2                [1, 100]                  --
│    └─FCBlock: 2-4                      --                        (recursive)
│    │    └─LeakyReLU: 3-3               [1, 100]                  --
│    └─FCBlock: 2-3                      [1, 100]                  --
│    │    └─Linear: 3-4                  [1, 100]                  10,100
│    │    └─Identity: 3-5                [1, 100]                  --
│    └─FCBlock: 2-4                      --                        (recursive)
│    │    └─LeakyReLU: 3-6               [1, 100]                  --
│    └─FCBlock: 2-5                      [1, 100]             

In [ ]:
# | export

class CoefficientsFC(nn.Module):
    """
    A wrapper around the FC class that outputs the coefficients of a filterbank
    """

    def __init__(
        self,
        n_parallel: int = 32,
        n_biquads: int = 2,
        initial_gain_scale: float = 1.0,
        tahn_c: float = 1.0,
        tanh_d: float = 1.0,
        use_zp_init: bool = False,
        initial_pole_mag: float = 1.5,
        initial_zero_mag: float = 0,
        **kwargs,
    ):
        super().__init__()
        self.n_parallel = n_parallel
        self.n_biquads = n_biquads
        self.initial_gain_scale = initial_gain_scale
        self.tahn_c = tahn_c
        self.tanh_d = tanh_d
        self.use_zp_init = use_zp_init

        if use_zp_init:
            z_offset = torch.polar(
                torch.zeros(self.n_parallel, self.n_biquads) * initial_zero_mag,
                torch.rand(self.n_parallel, self.n_biquads) * np.pi
            )
            p_offset = torch.polar(
                torch.ones(self.n_parallel, self.n_biquads) * initial_pole_mag,
                torch.rand(self.n_parallel, self.n_biquads) * np.pi
            )
            self.register_buffer("z_offset", z_offset)
            self.register_buffer("p_offset", p_offset)

        # override the output size
        kwargs["output_size"] = n_parallel * n_biquads * 5
        self.fc = FC(**kwargs)

    def forward(
        self,
        x: torch.Tensor, # A concatenated tensor of features, coodinates, and material properties
    ) -> torch.Tensor: # The concatenated coefficients of the filterbank
        
        x = self.fc(x)

        z, p, k = to_zpk(
            x=x,
            n_parallel=self.n_parallel,
            n_biquads=self.n_biquads,
            gain_factor=self.initial_gain_scale,
        )

        if self.use_zp_init:
            z = self.z_offset + z
            p = self.p_offset + p

        b = pole_or_zero_to_iir_coeff(z) * k[..., None]
        a = pole_or_zero_to_iir_coeff(
            constrain_complex_pole_or_zero(
                p,
                c=self.tahn_c,
                d=self.tanh_d,
            )
        )
    
        return torch.concat([b, a], dim=-1)

Test

In [ ]:
n_parallel = 32
n_biquads = 2
n_coeffs = 6
n_batches = 2
coefficients_fc = CoefficientsFC(
    input_size=1007,
    hidden_sizes=[100, 100],
    n_parallel=n_parallel,
    n_biquads=n_biquads,
    initial_gain_scale=1.0,
)

x = torch.randn(n_batches, 1007)
y = coefficients_fc(x)
assert y.shape == torch.Size([n_batches, n_parallel, n_biquads, n_coeffs])